# NFL Play Prediction in Any Down / Situation

We have had plenty of papers address when to go for it on 4th down, or the most optimal play to call on 3rd down. In this notebook, we will explore tendency analysis across the league and individual teams.

This notebook will go through the methodology of data collection, analysis, and results through multiple lenses. Once I explore the play-by-play data I have and any augmentations to the data, I will determine next steps and perhaps find a way to find some insights on the best plays to call given a game situation (i.e. 1st down, 2nd and short, 2nd and long, 3rd and short, 3rd and long, 4th). This will be quantified by the target variable of yards gained per play*. We may consider rushing and passing plays -- we do not consider fake punts / kicks.

*Or perhaps EPA? Does EPA take in post-snap information? Look into this further.

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
pd.set_option('display.max_columns', None)

## Data Scraping via nflfastR Package

We collect data for the past decade, 2010-2020. This is an arbitrary amount, but it reflects the shift of the NFL to a more pass-heavy league. Perhaps in subsamples of the data this will hold even more true, since there are considerable rule amendments that can affect the style of play each season. I used the following R code to pull the data. It was incredibly fast and easy to obtain via the nflFastR package and saved lots of time in scraping.

https://www.nflfastr.com/articles/nflfastR.html

```
require(nflfastR)
decade <- load_pbp(2010:2020)
write.csv(decade, file='nfl_pbp_2010-2020.csv')
```

This file is imported below, and is 983 MB. There is a considerable amount of data that can be chopped down to reduce the dimensionality of the data as well.

In [17]:
pbp = pd.read_csv('nfl_pbp_2010-2020.csv')
pbp = pbp.drop('Unnamed: 0', axis=1)

C:\Users\tshah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (46,180,181,183,184,190,191,194,195,198,199,204,205,206,207,210,211,214,215,219,220,221,223,225,227,234,235,236,237,238,239,249,250,254,255,256,261,263,264,267,268,269,270,284,285,295,296,302) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
pbp

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

We are provided with 372 columns in this data. We can supplement this dataset with offensive and defensive rankings for each team, win and score probability features, and PFF (Pro Football Focus) grades. I do not want to pay for a PFF subscription, so I will hold that thought for now and perhaps return to it later if fitted models do not perform well or could use additional information.

Before we do that, though, we must clean and augment our original data. First, we remove all features relating to individual player names or statistics on the play, since the intent is to do a team-based analysis. We also will look to remove post-snap features. Any information relating to the outcome of each play will defeat the purpose of our analysis when validating the models and thus the model will be useless if implemented in practice.

In [45]:
# drop columns all at once about:
## specific players
## timeouts
## jersey
to_drop_init = list(pbp.filter(regex='player')) + list(pbp.filter(regex='timeout')) + list(pbp.filter(regex='jersey'))
pbp_drop_init = pbp[pbp.columns.drop(to_drop_init)]

In [53]:
# after perusing other columns, I figured these were not important
to_drop = ['old_game_id', 'play_id', 'season_type', 'posteam_type',
           'week', 'side_of_field', 'game_date', 'quarter_end', 'time',
           'yrdln']
pbp = pbp_drop_init[pbp_drop_init.columns.drop(to_drop)]
pbp[pbp['play_type']=='pass']

,game_id,home_team,away_team,posteam,defteam,yardline_100,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,drive,sp,qtr,down,goal_to_go,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passing_yards,receiving_yards,rushing_yards,lateral_receiving_yards,lateral_rushing_yards,solo_tackle_1_team,solo_tackle_2_team,assist_tackle_1_team,assist_tackle_2_team,assist_tackle_3_team,assist_tackle_4_team,tackle_with_assist,tackle_with_assist_1_team,tackle_with_assist_2_team,fumbled_1_team,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_2_team,fumble_recovery_2_yards,return_team,return_yards,penalty_team,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,aborted_play,success,passer,rusher,receiver,pass,rush,first_down,special,play,passer_id,rusher_id,receiver_id,name,id,fantasy,fantasy_id,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
2,2010_01_ARI_STL,LA,ARI,ARI,LA,78.0,895.0,1795.0,3595.0,Half1,1.0,0,1,1.0,0,10,53.0,(14:55) 3-D.Anderson pass shor

In [51]:
# remove rows for kickoffs, FGs, punts, start of game (nan)
pbp['play_type'].unique()
pbp

array([nan, 'kickoff', 'pass', 'run', 'field_goal', 'punt', 'no_play',
       'extra_point', 'qb_kneel', 'qb_spike'], dtype=object)

# Features to Include

The main features I think will be useful in this model will be:

- Drive number
- Quarter of the game
- Down number
- Time left in a quarter
- Yards to gain for a first down
- Yards to gain for a touchdown
- Current score in the game (for the possessing team and defensive team, which would be modeled as a difference)
- Offensive Team
- Defensive Team
- Offensive Ranking thru Season (offensive team)
- Defensive Ranking thru Season (defensive team)
- Week of season
- Augment with PFF data? Aggregate individual player rankings?
- ...

Depending on the outcome I want to focus on, some of these may n

# Potential Outcomes

- Predict probability of first down
     - Can we judge whether a team will get a first down if they run or pass?
     - Which is more likely to result in success?
     - Would fit models separated by play type
- Predict yards gained per play
    - Similar problem to above but with a continuous outcome
    - Different methods and statistical techniques
    - Need to examine assumptions of each team
- Predict play type (run/pass) to predict playcalling tendencies
    - Goal is to see how predictable a team is with their play calling
    - Does being predictable in play calling (i.e. if a machine can learn their tendencies) lead to fewer wins?
    - Quantify how advantageous it is to be creative
    - Would need to fit models to each team

# Models

Depends on continous or categorical outcome..

# Preliminary Results